In [1]:
'''
tcloud中用销售订单生成税局发票模板,选择单个销售订单，单独生成相应的发票模板
'''
import os
import re
import easygui
import openpyxl
import numpy as np
import pandas as pd
import shutil

In [2]:
def danjuChuli(df):
    df = df.dropna(how='all', axis=0)
    cols = [i for i in df.columns.to_list() if 'Unnamed' not in i]
    df = df[cols]
    # get maxrows
    for label, ser in df.items():
        for num, x in enumerate(ser):
            if isinstance(x, str):
                if '合计' in x:
                    index = num
                    print(index)
                    break
    df = df.iloc[:index]
    df = df.dropna(how='all', axis=1)
    return df
def chuliMingchen(d):
    d['存货名称'] = d['存货名称'].str.replace('运费',"")
    qian = d['存货名称'].str.split('型').str[0]
    hou = d['存货名称'].str.split('型').str[1]
    qian = pd.Series(qian).fillna('')
    hou = pd.Series(hou).fillna('')
    return qian,hou

In [3]:
#读取发票模板
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
df_fapiao = pd.read_excel(fname_fapiao,header = 2,dtype = {'商品和服务税收分类编码':'str'})
df_fapiao

,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型


In [5]:
#多个订单
fname_dingdan = r"F:\repos\fish\销售订单 (4).xlsx"
df_dingdan = pd.read_excel(fname_dingdan,sheet_name = 0)
df_dingdan1 = danjuChuli(df_dingdan) 
qian,hou = chuliMingchen(df_dingdan1)
df_dingdan1

20


,单据执行状态,单据状态,单据编号,业务类型,冲抵订货,单据日期,创建时间,客户编码,客户,客户分类,...,行中止,累计采购数量,累计采购数量2,累计执行数量,累计执行数量2,未执行数量,未执行数量2,已采购需求分析,数据来源(明细),价格策略类型
0,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0039,普通销售,否,2024-08-08,2024-08-08 14:07:16,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,5000.0,25.0,5000,25.0,0,0.0,否,线下业务,存货批发价
1,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0039,普通销售,否,2024-08-08,2024-08-08 14:07:16,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,5000.0,25.0,5000,25.0,0,0.0,否,线下业务,存货批发价
2,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0039,普通销售,否,2024-08-08,2024-08-08 14:07:16,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,6000.0,30.0,6000,30.0,0,0.0,否,线下业务,NaN
3,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0039,普通销售,否,2024-08-08,2024-08-08 14:07:16,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,NaN,NaN,0,NaN,0,NaN,否,线下业务,NaN
4,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0051,普通销售,否,2024-08-13,2024-08-13 10:34:54,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,8000.0,40.0,8000,40.0,0,0.0,否,线下业务,存货批发价
5,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0051,普通销售,否,2024-08-13,2024-08-13 10:34:54,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,7000.0,35.0,7000,35.0,0,0.0,否,线下业务,存货批发价
6,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0051,普通销售,否,2024-08-13,2024-08-13 10:34:54,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,6000.0,30.0,6000,30.0,0,0.0,否,线下业务,存货批发价
7,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0051,普通销售,否,2024-08-13,2024-08-13 10:34:54,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,7000.0,35.0,7000,35.0,0,0.0,否,线下业务,NaN
8,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0051,普通销售,否,2024-08-13,2024-08-13 10:34:54,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,NaN,NaN,0,NaN,0,NaN,否,线下业务,NaN
9,已出库 已销货 未收订金 未开票 已转采购,生效,SO-2024-04-0062,普通销售,否,2024-08-15,2024-08-15 14:35:09,010197-004,肥猫音树贸易（武汉）有限公司,电商,...,否,7000.0,35.0,7000,35.0,0,0.0,否,线下业务,存货批发价


In [6]:
lst0 = df_dingdan1.columns.to_list()
lst1 = ['单据编号',
  '单据日期',
 '含税总金额',
 '存货名称',
 '存货分类',
 '存货编码',
  '存货代码',
  '数量',
  '件含量',
 '数量（件）',
  '含税单价',
 '含税金额',
        '客户'
]
df_dingdan1 = df_dingdan1[lst1]


In [7]:
df_dingdan1['项目名称'] = df_dingdan1['存货编码'] + '-' + hou
df_dingdan1['项目名称'] = df_dingdan1['项目名称'].str.replace('运费-','运费')
df_dingdan1['商品和服务税收分类编码'] = '1060202010000000000'
df_dingdan1['规格型号'] = qian
df_dingdan1['单位'] = '本'
df_dingdan1['商品数量'] = df_dingdan1['数量']
df_dingdan1['商品单价'] = df_dingdan1['含税单价']
df_dingdan1['金额'] = df_dingdan1['含税金额']
df_dingdan1['税率'] = 0
df_dingdan1['折扣金额'] = ''
df_dingdan1['优惠政策类型'] = ''

df_dingdan2 = df_dingdan1[['单据编号','项目名称',
 '商品和服务税收分类编码',
 '规格型号',
 '单位',
 '商品数量',
 '商品单价',
 '金额',
 '税率',
 '折扣金额',
 '优惠政策类型',
   '客户'                       ]]
df_dingdan2

C:\Users\redda\AppData\Local\Temp\ipykernel_31160\1009208513.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan1['项目名称'] = df_dingdan1['存货编码'] + '-' + hou
C:\Users\redda\AppData\Local\Temp\ipykernel_31160\1009208513.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan1['项目名称'] = df_dingdan1['项目名称'].str.replace('运费-','运费')
C:\Users\redda\AppData\Local\Temp\ipykernel_31160\1009208513.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

,单据编号,项目名称,商品和服务税收分类编码,规格型号,单位,商品数量,商品单价,金额,税率,折扣金额,优惠政策类型,客户
0,SO-2024-04-0039,N0781方格（电商）a-牛卡缝线语文本,1060202010000000000,16K60,本,5000,0.8,4000,0,,,肥猫音树贸易（武汉）有限公司
1,SO-2024-04-0039,N0783（电商）a-牛卡缝线英语本,1060202010000000000,16K60,本,5000,0.8,4000,0,,,肥猫音树贸易（武汉）有限公司
2,SO-2024-04-0039,N0785(5本）-牛卡缝线练习本,1060202010000000000,16K60,本,6000,0.8,4800,0,,,肥猫音树贸易（武汉）有限公司
3,SO-2024-04-0039,8888-,1060202010000000000,,本,0,NaN,-240,0,,,肥猫音树贸易（武汉）有限公司
4,SO-2024-04-0051,N0785（电商）a-牛卡缝线练习本,1060202010000000000,16K60,本,8000,0.8,6400,0,,,肥猫音树贸易（武汉）有限公司
5,SO-2024-04-0051,N0783（电商）a-牛卡缝线英语本,1060202010000000000,16K60,本,7000,0.8,5600,0,,,肥猫音树贸易（武汉）有限公司
6,SO-2024-04-0051,N0781方格（电商）a-牛卡缝线语文本,1060202010000000000,16K60,本,6000,0.8,4800,0,,,肥猫音树贸易（武汉）有限公司
7,SO-2024-04-0051,YS定制（混装）-牛卡缝线簿,1060202010000000000,16K60,本,7000,0.8,5600,0,,,肥猫音树贸易（武汉）有限公司
8,SO-2024-04-0051,8888-,1060202010000000000,,本,0,NaN,-420,0,,,肥猫音树贸易（武汉）有限公司
9,SO-2024-04-0062,N0785（电商）a-牛卡缝线练习本,1060202010000000000,16K60,本,7000,0.8,5600,0,,,肥猫音树贸易（武汉）有限公司


In [8]:
quan_jian = {'肥猫音树贸易（武汉）有限公司':'肥猫'}
fname_fapiao = r"F:\repos\fish\发票模板.xlsx"
path,file = os.path.split(fname_fapiao)
qian,suffix  = os.path.splitext(fname_fapiao)
gp = df_dingdan2.groupby('单据编号',group_keys = False)
for k,v in gp:
    gongsi = v.iloc[-1,-1]
    v  = v.iloc[:,1:-2]
    gongsi = quan_jian.get(gongsi,'待定')        #获取简称
    newname = "".join([qian,'-',gongsi,k,suffix])
    shutil.copyfile(fname_fapiao, newname)
    with pd.ExcelWriter(newname, engine='openpyxl',mode='a', if_sheet_exists='overlay')  as writer:
        v.to_excel(writer, sheet_name = '1-明细模板',startrow=3, header = False,index = False)


C:\Users\redda\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
